# AMR YAML Smoke Test (single subject)
Smoke test the AMR YAML configs without full batch runs. Follows grid-optimizer style but picks one subject from the YAML.

In [1]:
from __future__ import annotations

import sys
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd  # type: ignore[import]
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").is_dir():
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_ROOT = PROJECT_ROOT / "src"
DATA_ROOT = PROJECT_ROOT / "data" / "processed"

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
print(f"Data dir: {DATA_ROOT}")

Project root: D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning
Data dir: D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\data\processed


## Load YAML configuration and pick one subject

In [2]:
import yaml

# Choose which opt-config to test
config_name = "pmh_cond1"  # or "pmh_cond23"
config_path = PROJECT_ROOT / "configs" / "amr_opt_cfg" / f"{config_name}.yaml"

with config_path.open("r", encoding="utf-8") as f:
    opt_cfg = yaml.safe_load(f)

subjects = opt_cfg.get("subjects", [])
if not subjects:
    raise ValueError("No subjects found in the selected YAML")

subject_id = int(subjects[0])  # pick the first subject listed
engine_config = opt_cfg["engine_config"]
param_grid_yaml = opt_cfg.get("param_grid", {})
data_path = (config_path.parent / opt_cfg.get("data_path", "")).resolve()

print(f"Testing config: {config_path.name}")
print(f"Subjects in YAML: {subjects}")
print(f"Selected subject_id: {subject_id}")
print("Engine modules:")
pprint(engine_config.get("modules", {}))

Testing config: pmh_cond1.yaml
Subjects in YAML: [1, 4, 7, 10, 13, 16, 19, 22]
Selected subject_id: 1
Engine modules:
{'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule',
                          'kwargs': {'init_num': 3,
                                     'strategies': [{'amount': 'random_4',
                                                     'method': 'top_posterior'},
                                                    {'amount': 'opp_random_4',
                                                     'method': 'random'}]}},
 'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule',
                    'kwargs': {'beta': 'None'}},
 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule',
                'kwargs': {'gamma': 'None', 'w0': 'None'}},
 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule'}}


## Initialize StateModelAMROptimizer (test AMR mode)

In [3]:
from src.Bayesian_state.utils.state_amr_optimizer import StateModelAMROptimizer

# AMR hyperparams: use YAML if present, then override for a faster smoke test
amr_kwargs = opt_cfg.get("amr_kwargs", {}) or {}
test_amr_kwargs = {
    "max_evals": min(10, int(amr_kwargs.get("max_evals", 20))),
    "coarse_grid_per_dim": amr_kwargs.get("coarse_grid_per_dim", 3),
    "split_factor": amr_kwargs.get("split_factor", 2),
    "refine_top_k": amr_kwargs.get("refine_top_k", 2),
}

processed_data_dir = data_path.parent

optimizer = StateModelAMROptimizer(
    engine_config=engine_config,
    processed_data_dir=processed_data_dir,
    amr_kwargs=test_amr_kwargs,
    n_jobs=4,
)
print("AMR optimizer ready.")
print(f"AMR kwargs (test): {test_amr_kwargs}")

INFO:cat-learning:logger is running normally.


D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\logs\Run_20260106_215921.log
{'base_model': {'modules': {'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {}}}, 'agenda': ['likelihood_mod', '__self__']}, 'default_model': {'modules': {}, 'agenda': ['__self__']}, 'm_model': {'modules': {'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 15.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.8, 'gamma': 0.8}}}, 'agenda': ['likelihood_mod', 'memory_mod']}, 'pmh_model': {'modules': {'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule'}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule', 'kwargs': {'strategies': 'original_strategies_b', 'init_num': 8}}, 'likelihood_mod': {'class': 'src.Bayesi

## Small test grid and single-subject AMR run

In [4]:
# Start from YAML grid; optionally shrink for speed
param_grid = {k: list(v) for k, v in param_grid_yaml.items()}

# Take at most 2 values per dimension to keep it fast
param_grid = {k: v[:2] for k, v in param_grid.items()}

n_repeats = 1
refit_repeats = 2
window_size = int(opt_cfg.get("window_size", 16))
stop_at = float(opt_cfg.get("stop_at", 1.0))
max_trials_val = opt_cfg.get("max_trials", None)
max_trials = int(max_trials_val) if max_trials_val is not None else None

print(f"Running single-subject AMR test on subject {subject_id}")
print(f"Grid (truncated): {param_grid}")

results = optimizer.optimize_subject(
    subject_id=subject_id,
    param_grid=param_grid,
    n_repeats=n_repeats,
    refit_repeats=refit_repeats,
    window_size=window_size,
    stop_at=stop_at,
    max_trials=max_trials,
    keep_logs=False,
)

print("Done.")

Running single-subject AMR test on subject 1
Grid (truncated): {'gamma': [0.05, 0.95], 'w0': [0.01, 0.9], 'beta': [2.0, 32.0]}
Loading similarity matrix from disk: D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\cache\similarity_matrix_d4_c2.npy


INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'hypo_transitions_mod' registered as 'self.hypo_transitions_mod'.
INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\modules\memory.py:223: RuntimeWarning: divide by zero encountered in log
  return np.log(clipped)
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.


{'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06334599488144996, 0.07189273817979139], 'subject_id': 1}}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule', 'kwargs': {'strategies': [{'amount': 'random_4', 'method': 'top_posterior'}, {'amount': 'opp_random_4', 'method': 'random'}], 'init_num': 3}}, 'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 7.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.15833333333333335, 'gamma': 0.2}}}
name: perception_mod mod_kwargs: {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.055273313

INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'hypo_transitions_mod' registered as 'self.hypo_transitions_mod'.
INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\modules\memory.py:223: RuntimeWarning: divide by zero encountered in log
  return np.log(clipped)
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.


{'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06334599488144996, 0.07189273817979139], 'subject_id': 1}}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule', 'kwargs': {'strategies': [{'amount': 'random_4', 'method': 'top_posterior'}, {'amount': 'opp_random_4', 'method': 'random'}], 'init_num': 3}}, 'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 2.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.010000000000000009, 'gamma': 0.05000000000000002}}}
name: perception_mod mod_kwargs: {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.0186535324675

...

{'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06334599488144996, 0.07189273817979139], 'subject_id': 1}}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule', 'kwargs': {'strategies': [{'amount': 'random_4', 'method': 'top_posterior'}, {'amount': 'opp_random_4', 'method': 'random'}], 'init_num': 3}}, 'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 12.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.9, 'gamma': 0.95}}}
name: perception_mod mod_kwargs: {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06

INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'hypo_transitions_mod' registered as 'self.hypo_transitions_mod'.
INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\modules\memory.py:223: RuntimeWarning: divide by zero encountered in log
  return np.log(clipped)
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'hypo_transitions_mod' registered as 'self.hypo_transitions_mod'.
INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\modules\memory.py:223: RuntimeWarning: divide by zero encountered in log
  return np.log(clipped)
INFO:ca

{'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06334599488144996, 0.07189273817979139], 'subject_id': 1}}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule', 'kwargs': {'strategies': [{'amount': 'random_4', 'method': 'top_posterior'}, {'amount': 'opp_random_4', 'method': 'random'}], 'init_num': 3}}, 'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 22.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.9, 'gamma': 0.95}}}
name: perception_mod mod_kwargs: {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06

INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'hypo_transitions_mod' registered as 'self.hypo_transitions_mod'.
INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\modules\memory.py:223: RuntimeWarning: divide by zero encountered in log
  return np.log(clipped)
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:Refitting best params for subject 1 with 2 repeats.


{'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06334599488144996, 0.07189273817979139], 'subject_id': 1}}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.DynamicHypothesisModule', 'kwargs': {'strategies': [{'amount': 'random_4', 'method': 'top_posterior'}, {'amount': 'opp_random_4', 'method': 'random'}], 'init_num': 3}}, 'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 32.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.9, 'gamma': 0.95}}}
name: perception_mod mod_kwargs: {'mean': [0.004842201028895644, 0.0287070643238552, 0.04740832529649071, 0.05244638105800858], 'std': [0.01865353246756878, 0.05527331384069292, 0.06

## Summarize best result and grid dataframe

In [5]:
best = results["best"]
print("=" * 40)
print(f"Best params: {best.params}")
print(f"Mean error: {best.mean_error:.6f}")
print(f"Std error:  {getattr(best, 'std_error', 0.0):.6f}")
print("=" * 40)

# Build dataframe for inspection
grid_rows = []
for res in results["grid"]:
    row = res.params.copy()
    row["mean_error"] = res.mean_error
    row["std_error"] = getattr(res, "std_error", np.nan)
    grid_rows.append(row)

df_results = pd.DataFrame(grid_rows).sort_values("mean_error")
print(df_results.head())

Best params: {'gamma': 0.95, 'w0': 0.30666666666666675, 'beta': 22.0}
Mean error: 0.055952
Std error:  0.006839
    gamma        w0  beta  mean_error  std_error
95   0.95  0.306667  22.0    0.059163        0.0
75   0.65  0.900000  22.0    0.059301        0.0
47   0.05  0.900000  32.0    0.060558        0.0
48   0.35  0.900000  32.0    0.068737        0.0
85   0.95  0.306667  22.0    0.069385        0.0


## Quick diagnostic plot (optional)

In [7]:
# Heatmap if 2D grid (gamma x w0)
if {"gamma", "w0"}.issubset(df_results.columns):
    pivot = df_results.pivot(index="gamma", columns="w0", values="mean_error")
    plt.figure(figsize=(6, 4))
    sns.heatmap(pivot, annot=True, fmt=".3f", cmap="viridis_r")
    plt.title(f"Error heatmap (subject {subject_id})")
    plt.show()

# Accuracy curve for best result (if metrics available)
metrics = best.metrics
if "sliding_true_acc" in metrics and "sliding_pred_acc" in metrics:
    trials = np.arange(len(metrics["sliding_true_acc"]))
    plt.figure(figsize=(10, 4))
    plt.plot(trials, metrics["sliding_true_acc"], label="subject", color="black")
    plt.plot(trials, metrics["sliding_pred_acc"], label="model", color="red")
    plt.ylim(0, 1.05)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.title(f"Accuracy curve (subject {subject_id})")
    plt.legend()
    plt.show()


ValueError: Index contains duplicate entries, cannot reshape